In [1]:
import sscps
from simple_dmet import *
from pyscf.gto import mole
from pyscf import scf, lo, fci, ao2mo
import numpy as np


def make_Hring(l=1.):
    r = l/(2 * np.sin(np.pi / 10))
    atmlst = []
    for i in range(10):
        atmlst.append(['H', (r*np.cos(2. * np.pi/10*i), r*np.sin(2. * np.pi/10*i), 0)])
    mol = gto.Mole()
    mol.atom = atmlst
    mol.basis = 'sto-6g'
    mol.build()
    return mol

def calc(mol:gto.Mole):

    m = scf.RHF(mol); m.kernel()
    cal = lo.orth_ao(mol)
    clm = np.dot(np.linalg.inv(cal), m.mo_coeff)[:,np.argsort(m.mo_energy)[:mol.nelec[0]]]

    ci_slater = sscps.ci_slater(mol.nao, mol.nelec, clm)
    
    dmetscf = DMET_SCF(m, [0], mu_glob=0.05)

    
    def f(mu_glob):
        dmetscf.change_mu_glob(mu_glob)
        e, nx = dmetscf.rhf()
        return nx-1
    mu_glob, count = secant(f, 0, 0.1, 1E-12)

    mo_e, mo_c = dmetscf.mo_energy, dmetscf.mo_coeff
    c = mo_c[:,np.argsort(mo_e)[0]]

    coef_hf = np.einsum("i,j->ij", c, c)
    
    def g(mu_glob):
        dmetscf.change_mu_glob(mu_glob)
        e, nx = dmetscf.fci()
        return nx-1
    mu_glob, count = secant(g, 0, 0.1, 1E-12)

    cis = fci.direct_spin1.FCISolver(mol)
    e, coef_fci = cis.kernel(dmetscf.get_hcore(), dmetscf.erifb, 2, 2)

    proj = (coef_fci / coef_hf).reshape(-1) 

    ci = sscps.ci_sscps(mol.nao, mol.nelec, ci_slater, proj)
    
    eri = ao2mo.kernel(mol, cal)
    h1e = np.einsum("mj,mn,nk->jk", cal.conj(), scf.hf.get_hcore(mol), cal)

    e = fci.direct_spin1.energy(h1e, eri, ci, mol.nao, mol.nelec)
        
    print("Energy = ", e + mol.energy_nuc())
    return e + mol.energy_nuc()


In [2]:
import matplotlib.pyplot as plt

bllst = np.linspace(0.5, 2.5, 21)
elst = []
for bl in bllst:
    elst.append(calc(make_Hring(bl)))


converged SCF energy = -2.87405396382651
Number of Fragment Orbitals: 1
True


<class 'simple_dmet.DMET_SCF'> does not have attributes  dm mu_glob un_dm nocc un_coeff na erifb fb_coeff


converged SCF energy = -2.81383003089615
converged SCF energy = -2.85412396889537
converged SCF energy = -2.81383003089317
Norm =  0.9999999999999988
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
Energy =  -2.891650327662859
converged SCF energy = -4.21118796298111
Number of Fragment Orbitals: 1
True
converged SCF energy = -2.52647265839179
converged SCF energy = -2.58528203211732
converged SCF energy = -2.5264726583902
Norm =  0.999999999999999
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
Energy =  -4.235601958409198
converged SCF energy = -4.86581202324304
Number of Fragment Orbitals: 1
True
converged SCF energy = -2.2911702269757
converged SCF energy = -2.36692554904564
converged SCF energy = -2.29117022697467
Norm =  

In [1]:
# calculate <\Phi|G^{-1}HG|\Phi> instead of variational energy

import sscps
from simple_dmet import *
from pyscf.gto import mole
from pyscf import scf, lo, fci, ao2mo
import numpy as np
from pyscf.fci.direct_spin1 import absorb_h1e, contract_2e

def ovlp(h1e, eri, ci, ciinv, norb, nelec, link_index=None):
    h2e = absorb_h1e(h1e, eri, norb, nelec, .5)
    ci1 = contract_2e(h2e, ci, norb, nelec, link_index)
    return np.dot(ciinv.reshape(-1), ci1.reshape(-1))


def make_Hring(l=1.):
    r = l/(2 * np.sin(np.pi / 10))
    atmlst = []
    for i in range(10):
        atmlst.append(['H', (r*np.cos(2. * np.pi/10*i), r*np.sin(2. * np.pi/10*i), 0)])
    mol = gto.Mole()
    mol.atom = atmlst
    mol.basis = 'sto-6g'
    mol.build()
    return mol

def calc(mol:gto.Mole):

    m = scf.RHF(mol); m.kernel()
    cal = lo.orth_ao(mol)
    clm = np.dot(np.linalg.inv(cal), m.mo_coeff)[:,np.argsort(m.mo_energy)[:mol.nelec[0]]]

    ci_slater = sscps.ci_slater(mol.nao, mol.nelec, clm)
    
    dmetscf = DMET_SCF(m, [0], mu_glob=0.05)

    
    def f(mu_glob):
        dmetscf.change_mu_glob(mu_glob)
        e, nx = dmetscf.rhf()
        return nx-1
    mu_glob, count = secant(f, 0, 0.1, 1E-12)

    mo_e, mo_c = dmetscf.mo_energy, dmetscf.mo_coeff
    c = mo_c[:,np.argsort(mo_e)[0]]

    coef_hf = np.einsum("i,j->ij", c, c)
    
    def g(mu_glob):
        dmetscf.change_mu_glob(mu_glob)
        e, nx = dmetscf.fci()
        return nx-1
    mu_glob, count = secant(g, 0, 0.1, 1E-12)

    cis = fci.direct_spin1.FCISolver(mol)
    e, coef_fci = cis.kernel(dmetscf.get_hcore(), dmetscf.erifb, 2, 2)

    proj = (coef_fci / coef_hf).reshape(-1) 

    projinv = 1. / proj # since in 1H situ, P^{-1} = \sum\limits_i p_i^{-1} | i \rangle \langle i |

    ci = sscps.ci_sscps(mol.nao, mol.nelec, ci_slater, proj, normalized=False)
    ciinv = sscps.ci_sscps(mol.nao, mol.nelec, ci_slater, projinv, normalized=False)   
    eri = ao2mo.kernel(mol, cal)
    h1e = np.einsum("mj,mn,nk->jk", cal.conj(), scf.hf.get_hcore(mol), cal)

    e = ovlp(h1e, eri, ci, ciinv, mol.nao, mol.nelec)
        
    print("Energy = ", e + mol.energy_nuc())
    return e + mol.energy_nuc()

import matplotlib.pyplot as plt

bllst = np.linspace(0.5, 2.5, 21)
elst = []
for bl in bllst:
    elst.append(calc(make_Hring(bl)))

np.savetxt("e-2H-st.txt", np.array((bllst, elst)))

converged SCF energy = -2.87405396382649


<class 'simple_dmet.DMET_SCF'> does not have attributes  cal fb_coeff nocc un_dm na erifb dm mu_glob frag_coeff bath_coeff un_coeff


converged SCF energy = -2.81383003089614
converged SCF energy = -2.85412396889536
converged SCF energy = -2.81383003089317
Norm =  0.9999999999999983
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
Norm =  1.001019319216736
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
Energy =  -2.916974252321669
converged SCF energy = -4.21118796298111
converged SCF energy = -2.52647265839179
converged SCF energy = -2.58528203211732
converged SCF energy = -2.5264726583902
Norm =  0.9999999999999983
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
allclose =  False
Norm =  1.0019333153900218
allclose =  False
allclose =  False
al